In [1]:
import psycopg2
from psycopg2 import sql
from sqlalchemy import create_engine
import pandas as pd
import numpy as np

In [2]:
# Replace these with your database details
dbname = "moodmap_db"
user = "postgres"
password = "9b6b09d4a1"
host = "localhost"
port = "5432"

# Create a connection string
conn_str = f"postgresql://{user}:{password}@{host}:{port}/{dbname}"

# Create a connection
engine = create_engine(conn_str)


In [ ]:
    # total_calls,                            |  +
    # satisfied_calls,                        | will add to db or calculate here
    # not_satisfied_calls,                    | will add to db or calculate here
    # alerts,                                 |  +
    # customer_sat_level,                     |  +  
    # agent_perf_rate,                        |  +
    # costumer_care,                          |  +

    # average_customer_voice_emotions,
    # average_agent_voice_emotions,
    # average_customer_text_emotions,
    # average_agent_text_emotions,
    # share_of_customer_voice_emotion_timeline,
    # share_of_agent_voice_emotion_timeline,
    # share_of_customer_text_emotion_timeline,
    # share_of_agent_text_emotion_timeline,

    # call_topics,                            |  +
    # ironic_count,                           |  +
    # not_ironic_count,                       |  +
    # hateful_count,                          |  +
    # targeted_count,                         |  - 
    # aggressive_count,                       |  -

    # Normal                                  |  +

In [3]:
def get_data_info(start_date,end_date,satisfaction_level,performance_level):
    # Replace this with your SQL query
    query_call = """SELECT * FROM public.call
                            WHERE starttime > '{s_d}' and endtime < '{e_d}'
                            and customer_satisfaction_rate >'{c_s_0}' and customer_satisfaction_rate <'{c_s_1}';""".format(s_d=start_date,e_d=end_date,c_s_0=satisfaction_level[0],c_s_1=satisfaction_level[1])

    # Replace this with your SQL query
    query_call_info = """SELECT * FROM public.callinfo"""

    df_call = pd.read_sql_query(query_call, engine)
    df_call_info = pd.read_sql_query(query_call_info, engine)

    df = pd.merge(df_call,df_call_info,how='left',on='callid')

    # Replace this with your SQL query
    query_clint_voice_emotion = """
                        SELECT * FROM public.call_mood_client
                        WHERE callid IN {}
                            """.format(tuple(df.callid))

    clint_voice_emotion = pd.read_sql_query(query_clint_voice_emotion, engine)

    # Replace this with your SQL query
    query_agent_voice_emotion = """
                        SELECT * FROM public.call_mood_agent
                        WHERE callid IN {}
                            """.format(tuple(df.callid))

    agent_voice_emotion = pd.read_sql_query(query_agent_voice_emotion, engine)

    # Replace this with your SQL query
    query_clint_text_emotion = """
                        SELECT * FROM public.call_sentence_client
                        WHERE callid IN {}
                            """.format(tuple(df.callid))

    clint_text_emotion = pd.read_sql_query(query_clint_text_emotion, engine)

    # Replace this with your SQL query
    query_agent_text_emotion = """
                        SELECT * FROM public.call_sentence_agent
                        WHERE callid IN {}
                            """.format(tuple(df.callid))

    agent_text_emotion = pd.read_sql_query(query_agent_text_emotion, engine)

       # Extract emotions and counts
    voice_emotions_clinet = pd.DataFrame(clint_voice_emotion.emotion.value_counts()).reset_index()['emotion'].tolist()
    voice_counts_client = pd.DataFrame(clint_voice_emotion.emotion.value_counts()).reset_index()['count'].tolist()

    # Extract emotions and counts
    voice_emotions_agent = pd.DataFrame(agent_voice_emotion.emotion.value_counts()).reset_index()['emotion'].tolist()
    voice_counts_agent = pd.DataFrame(agent_voice_emotion.emotion.value_counts()).reset_index()['count'].tolist()

    # Extract emotions and counts
    text_emotions_clinet = pd.DataFrame(clint_text_emotion.sentiment.value_counts()).reset_index()['sentiment'].tolist()
    text_counts_client = pd.DataFrame(clint_text_emotion.sentiment.value_counts()).reset_index()['count'].tolist()

    # Extract emotions and counts
    text_emotions_agent = pd.DataFrame(agent_text_emotion.sentiment.value_counts()).reset_index()['sentiment'].tolist()
    text_counts_agent = pd.DataFrame(agent_text_emotion.sentiment.value_counts()).reset_index()['count'].tolist()

    tatal_calls = len(df)
    alerts = df.alert.value_counts()[1]
    customer_sat_level = df.customer_satisfaction_rate.mean()
    agent_perf_rate = df.agent_performance_rate.mean()
    costumer_care = df.callscore.mean()
    call_topics = df[['topic']]
    ironic_count = np.where(df.ironypercent > 30,1,0).sum()
    not_ironic_count = (1 - np.where(df.ironypercent > 30,1,0)).sum()
    hateful_count = np.where(df.hatespeechpercent > 30,1,0).sum()
    not_hateful_count = (1 - np.where(df.hatespeechpercent > 30,1,0)).sum()

    return voice_emotions_clinet,voice_counts_client,voice_emotions_agent,voice_counts_agent,text_emotions_clinet,text_counts_client,text_emotions_agent,text_counts_agent,tatal_calls,alerts,customer_sat_level,agent_perf_rate,costumer_care,call_topics,ironic_count,not_ironic_count,hateful_count,not_hateful_count

In [4]:
# Replace this with your SQL query
query_call = """SELECT * FROM public.call;"""
# Replace this with your SQL query
query_call_info = """SELECT * FROM public.callinfo"""
df_call = pd.read_sql_query(query_call, engine)
df_call_info = pd.read_sql_query(query_call_info, engine)
df = pd.merge(df_call,df_call_info,how='left',on='callid')

In [7]:
# Replace this with your SQL query
query_clint_voice_emotion = """
                    SELECT * FROM public.call_mood_client
                    WHERE callid IN {}
                        """.format(tuple(df.callid))
clint_voice_emotion = pd.read_sql_query(query_clint_voice_emotion, engine)
# Replace this with your SQL query
query_agent_voice_emotion = """
                    SELECT * FROM public.call_mood_agent
                    WHERE callid IN {}
                        """.format(tuple(df.callid))
agent_voice_emotion = pd.read_sql_query(query_agent_voice_emotion, engine)


In [8]:

   # Extract emotions and counts
voice_emotions_clinet = pd.DataFrame(clint_voice_emotion.emotion.value_counts()).reset_index()['emotion'].tolist()
voice_counts_client = pd.DataFrame(clint_voice_emotion.emotion.value_counts()).reset_index()['count'].tolist()
# Extract emotions and counts
voice_emotions_agent = pd.DataFrame(agent_voice_emotion.emotion.value_counts()).reset_index()['emotion'].tolist()
voice_counts_agent = pd.DataFrame(agent_voice_emotion.emotion.value_counts()).reset_index()['count'].tolist()
# Extract emotions and counts

In [11]:
# Replace this with your SQL query
query_clint_text_emotion = """
                    SELECT * FROM public.call_sentence_client
                    WHERE callid IN {}
                        """.format(tuple(df.callid))
clint_text_emotion = pd.read_sql_query(query_clint_text_emotion, engine)
# Replace this with your SQL query
query_agent_text_emotion = """
                    SELECT * FROM public.call_sentence_agent
                    WHERE callid IN {}
                        """.format(tuple(df.callid))
agent_text_emotion = pd.read_sql_query(query_agent_text_emotion, engine)

In [12]:
# Extract emotions and counts
text_emotions_clinet = pd.DataFrame(clint_text_emotion.sentiment.value_counts()).reset_index()['sentiment'].tolist()
text_counts_client = pd.DataFrame(clint_text_emotion.sentiment.value_counts()).reset_index()['count'].tolist()


In [13]:
agent_text_emotion

,id,callid,speechtext,sentiment,irony,starttime,endtime,hatelevelpercent,meanemotion
0,1,401,Random text 1,neutral,0,2024-01-13 18:18:27.824429,2024-01-13 17:54:37.138510,57,happy
1,2,402,Random text 2,negative,1,2024-01-13 18:24:25.589352,2024-01-13 18:05:48.456728,26,sad
2,3,403,Random text 3,neutral,1,2024-01-13 18:01:53.055021,2024-01-13 18:20:46.633131,73,neutral
3,4,404,Random text 4,positive,0,2024-01-13 17:44:33.903565,2024-01-13 18:06:54.104108,71,sad
4,5,405,Random text 5,positive,0,2024-01-13 17:57:39.193944,2024-01-13 17:44:45.626066,19,neutral
...,...,...,...,...,...,...,...,...,...
95,96,496,Random text 96,negative,0,2024-01-13 18:08:05.094322,2024-01-13 18:10:10.022695,85,sad
96,97,497,Random text 97,positive,0,2024-01-13 17:55:21.196371,2024-01-13 17:34:38.393578,4,neutral
97,98,498,Random text 98,neutral,1,2024-01-13 17:58:17.081540,2024-01-13 17:36:39.449575,57,sad
98,99,499,Random text 99,neutral,0,2024-01-13 18:20:33.511802,2024-01-13 18:23:48.743449,58,sad
